# ***CineClassify***

In dit notebook werken we aan onze opdracht van CineClassify. In dit notebook zal er een pipeline worden gebouwd om de data in te laden en er een dataframe van te maken. Deze dataframe kan worden gebruikt om de data duidelijk in te zien voor het datascience team.

In [1]:
# Importeren standaard libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importeren webscraping libaries
import requests
from bs4 import BeautifulSoup
import regex as re

# Importeren time-out libaries
from time import sleep
from random import randint

## **IMDb Webscraping**

Om een deel van de data te krijgen is het nodig om data te verkrijgen van het internet. Dit wordt gedaan door middel van een techniek genaamd webscraping. Door middel van de BeautifullSoup library voor Python is het gemakkelijk gemaakt om deze stappen te ondernemen. Door het bekijken van de HTML code van de website kunnen de nodige elementen gevonden worden en kan de data opgehaald worden van de website. Voordat we beginnen met het coderen van de soup worden er eerst een paar nodige elementen aangemaakt.

In [2]:
# Aanmaken van lijsten om de data in te stoppen
titel = []
jaartal = []
lengte = []
imdb_scores = []
meta_scores = []
stemmen = []
us_omzet = []
omschrijving = []
certificaat = []
genre = []
regisseur = []
sterren = []


# Verkrijgen van de engelse namen van films
en_titel = {'Accept-Language': 'en-US, en;q=0.5'}

# Aanmaken van lijst voor pagina's
pagina = np.arange(1, 1001, 50)

# Aanmaken van de URL
url = ('https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&start=')

Nu de basis is aangemaakt kan er begonnen worden met het aanmaken en gebruiken van de soup.

```py
# Zorgen dat de scraping voor elke 50 gaat
for p in pagina:
    # Pakken van URL
    p = requests.get(
        url + str(p) + '&ref_=adv_nxt', headers=en_titel
        )

    # Beginnen van de soup
    soup = BeautifulSoup(p.text, 'html.parser')

    # Zoeken van alle films op de pagina
    films = soup.find_all('div', class_='lister-item mode-advanced')

    # Wachtijd van 2 tot 10 seconden
    sleep(randint(2, 10))

    for item in films:
        # Titel
        titel.append(item.h3.a.text)

        # Jaartal
        jaartal.append(item.h3.find('span', class_='lister-item-year').text)

        # Regisseur
        regisseur.append(item.find('p', class_='').find('a').text)

        # Hoofd-acteurs
        acteurs = item.find('p', class_='').find_all('a')
        stars = []
        for tag in acteurs[-4:]:
            stars.append(tag.text)
        sterren.append(stars)

        # Leeftijd certificatie
        cert = (item.find('span', class_='certificate').text
                if item.p.find('span', class_='certificate') else 'NotFound')
        certificaat.append(cert)

        # Lengte
        runtime = (item.find('span', class_='runtime').text
                    if item.p.find('span', class_='runtime') else 'NotFound')
        lengte.append(runtime)

        # Genre
        gen = (item.find('span', class_='genre').text
                if item.p.find('span', class_='genre') else 'NotFound')
        genre.append(gen)

        # IMDb rating
        imdb_scores.append(float(item.strong.text))

        # meta_scores
        m_score = (item.find('span', class_='metascore').text
                    if item.find('span', class_='metascore') else 'NotFound')
        meta_scores.append(m_score)

        # Omschrijving
        desc = item.find_all('p', class_='text-muted')
        omschrijving.append(desc[1].text)

        # Stemmen en Omzet
        so = item.find_all('span', attrs={'name':'nv'})
        stemmen.append(so[0].text)
        us_omzet.append(so[1].text if len(so) > 1 else '-')

    print("-- Iteratie van loop voltooid --")

```

Nu alle lijsten gevuld zijn met de data van de IMDb Top 1000 films kan het in een dataframe gezet worden voor opschoning.

```py
# Aanmaken van een dataframe
films = pd.DataFrame(
    {'Film' : titel,
     'Omschrijving' : omschrijving,
     'Regisseur' : regisseur,
     'Hoofd Acteurs' : sterren,
     'Age_Rating' : certificaat,
     'Genres' : genre,
     'Jaar' : jaartal,
     'Minuten' : lengte,
     'IMDb_Score' : imdb_scores,
     'Meta_Score' : meta_scores,
     'Stemmen' : stemmen,
     'Omzet (in M)' : us_omzet}
)

# Tonen van de eerste 10 regels
films.head(10)
```

## **Opschonen van films DataFrame**

Voordat de data actief wordt opgeschoond wordt de informatie over het DataFrame bekeken.

```py
# Bekijken van info over het dataframe
display(films.info())

# Opschonen van de omschrijving kolom
films['Omschrijving'] = films['Omschrijving'].str.strip()

# Opschonen van de acteurs kolom
films['Hoofd Acteurs'] = films['Hoofd Acteurs'].astype(str)\
                            .replace({'\'': '', '\[|\]': ''}, regex=True)

# Opschonen van de genres kolom
films['Genres'] = films['Genres'].str.strip()

# Opschonen van de Jaar kolom
films['Jaar'] = films['Jaar'].str.extract('(\d+)').astype(int)

# Opschonen van de Minuten kolom
films['Minuten'] = films['Minuten'].str.extract('(\d+)').astype(int)

# Opschonen van de Meta_score kolom
films['Meta_Score'] = films['Meta_Score'].str.extract('(\d+)')

# Omzetten naar float en NotFound veranderen naar NaN
films['Meta_Score'] = pd.to_numeric(films['Meta_Score'], errors='coerce')

# Opschonen van de Stemmen kolom
films['Stemmen'] = films['Stemmen'].str.replace(',', '').astype(int)

# Opschonen van Omzet kolom
# Weghalen van '$' en 'M'
films['Omzet (in M)'] = films['Omzet (in M)'].map(lambda x: x.lstrip('$').rstrip('M'))

# Omzetten naar float en NotFound veranderen naar NaN
films['Omzet (in M)'] = pd.to_numeric(films['Omzet (in M)'], errors='coerce')

# Tonen van de eerste 10 regels na aanpassingen
films.head(10)

# Kijken of alle data typen succesvol zijn veranderd
display(films.dtypes)
display(films)

films.to_csv('IMDb_data.csv')
```

In [3]:
# Inladen van IMDb Data
films = pd.read_csv('IMDb_data.csv', index_col=0)

# Tonen data
display(films.dtypes)
display(films)

Film              object
Omschrijving      object
Regisseur         object
Hoofd Acteurs     object
Age_Rating        object
Genres            object
Jaar               int64
Minuten            int64
IMDb_Score       float64
Meta_Score       float64
Stemmen            int64
Omzet (in M)     float64
dtype: object

,Film,Omschrijving,Regisseur,Hoofd Acteurs,Age_Rating,Genres,Jaar,Minuten,IMDb_Score,Meta_Score,Stemmen,Omzet (in M)
0,The Shawshank Redemption,"Over the course of several years, two convicts...",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",R,Drama,1994,142,9.3,82.0,2818589,28.34
1,The Godfather,"Don Vito Corleone, head of a mafia family, dec...",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",R,"Crime, Drama",1972,175,9.2,100.0,1964393,134.97
2,The Dark Knight,When the menace known as the Joker wreaks havo...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",PG-13,"Action, Crime, Drama",2008,152,9.0,84.0,2800102,534.86
3,Schindler's List,"In German-occupied Poland during World War II,...",Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro...",R,"Biography, Drama, History",1993,195,9.0,95.0,1416792,96.90
4,The Lord of the Rings: The Return of the King,Gandalf and Aragorn lead the World of Men agai...,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",PG-13,"Action, Adventure, Drama",2003,201,9.0,94.0,1930207,377.85
...,...,...,...,...,...,...,...,...,...,...,...,...
995,When Marnie Was There,"Due to 12-year-old Anna's asthma, she's sent t...",Hiromasa Yonebayashi,"Sara Takatsuki, Kasumi Arimura, Nanako Matsush...",PG,"Animation, Drama, Family",2014,103,7.6,72.0,45579,0.77
996,Control,"A profile of Ian Curtis, the enigmatic singer ...",Anton Corbijn,"Sam Riley, Samantha Morton, Craig Parkinson, A...",R,"Biography, Drama, Music",2007,122,7.6,78.0,67786,0.87
997,Philomena,A world-weary political journalist picks up th...,Stephen Frears,"Judi Dench, Steve Coogan, Sophie Kennedy Clark...",PG-13,"Biography, Comedy, Drama",2013,98,7.6,77.0,102917,37.71
998,Shine,"Pianist David Helfgott, driven by his father a...",Scott Hicks,"Geoffrey Rush, Armin Mueller-Stahl, Justin Bra...",PG-13,"Biography, Drama, Music",1996,105,7.6,87.0,55997,35.81


## **Corpus Film Omschrijvingen**

De open data bron bevat verschillende informatie van ongeveer 42000 films. In deze bron staat informatie over de verschillende karakters in de films, een uitgebreide omschrijving van het plot en metadata over de film. Voor dit onderzoek zal de data over de karakters niet nodig zijn en dus zal deze niet worden ingeladen. Hieronder zijn de stappen beschreven en vervolgens is er een functie aangemaakt die ervoor zorgt dat de data wordt ingeladen en samengevoegd. Met dank aan de README.txt die bij de download van de data zat, is het gemakkelijker om de data en de structuur te begrijpen. Voor het inladen van elk bestand is de README gebruikt om de kolomnamen aan te maken en de data op de juiste manier in te laden. Bij de kopjes voor het inladen is de tekst erbij geplakt.

#### **Bestand: plot_summaries.txt**

Door middel van pd.read_csv is het mogelijk om text bestanden in te lezen. Bij het bekijken van het tekst bestand werd duidelijk dat de scheiding van kolommen is aangegeven door een tab. Verder is de README informatie gebruikt om de kolomnamen te maken:
- Plot summaries of 42,306 movies extracted from the November 2, 2012 dump of English-language Wikipedia.  Each line contains the Wikipedia movie ID (which indexes into movie.metadata.tsv) followed by the summary.

#### **Bestand: movie.metadata.tsv**

Ook bij dit tsv bestand kan er gebruik worden gemaakt van de pd.read_csv functie. Bij het lezen van de README is duidelijk neergezet hoe de structuur van dit bestand eruit ziet.

Metadata for 81,741 movies, extracted from the Noverber 4, 2012 dump of Freebase.  Tab-separated; columns:

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie name
4. Movie release date
5. Movie box office revenue
6. Movie runtime
7. Movie languages (Freebase ID:name tuples)
8. Movie countries (Freebase ID:name tuples)
9. Movie genres (Freebase ID:name tuples)

#### **Joinen van de DataFrames**

Doordat beide databronnen een Wikipedia_ID hebben, is het gemakkelijk om deze same te voegen. Om dit te doen zal de .merge() method gebruikt worden van pandas. Tijdens het merge wordt er gebruik gemaakt van de how parameter, om te zorgen dat alleen rijen waarvan er een omschrijving is aanwezig zijn.

In [2]:
def corpus_load():
    """
    Deze functie laad de nodige corpus data in.

    Returns:
    ----------
    df : pandas.DataFrame
        Dit dataframe bevat alle data die is ingeladen.
        De kolommen bestaan uit ID nummers, Naam,
        release date, omzet, lengte, talen, landen,
        genres en omschrijvingen.
    """

    # Inladen databronnen
    film_meta = pd.read_csv('movie.metadata.tsv', header=None, sep='\t')
    plot_sum = pd.read_csv('plot_summaries.txt', header=None, sep='\t')

    # Aanmaken kolomnamen
    film_meta.columns = ['Wikipedia_ID', 'Freebase_ID', 'MovieName', 'ReleaseDate',
                        'BoxOffice', 'Lengte', 'Talen', 'Landen', 'Genres']
    plot_sum.columns = ['Wikipedia_ID', 'Summary']

    # Selecteren van de nodige kolommen
    film_data = film_meta[['Wikipedia_ID', 'MovieName',
                           'Lengte', 'Genres']]
    
    # Samenvoegen van de data
    df = plot_sum.merge(film_data,
                            how='left',
                            left_on='Wikipedia_ID',
                            right_on='Wikipedia_ID')
    
    return df

In [3]:
corpus = corpus_load()
corpus.to_csv('corpus.csv', index=False)

## Data transformeren naar SQL database

Om te voldoen aan de eisen van de opgave hebben wij een SQL Database nodig. Via de website freeSQLdatabase.com hebben we een database kunnen opstellen om aan de eisen te voldoen. In het volgende blok zorgen wij ervoor dat er data op de server wordt gezet.

In [10]:
import mysql.connector

# Replace these values with your actual database information
host = "sql11.freesqldatabase.com"
database_name = "sql11667769"
database_user = "sql11667769"
database_password = "rMWK4j1uCv"
port_number = "3306"

# Connection to MySQL
connection = mysql.connector.connect(
    host=host,
    database=database_name,
    user=database_user,
    password=database_password,
    port=port_number
)

# Create a cursor
cursor = connection.cursor()

# Read CSV file into a DataFrame
csv_file_path = "corpus.csv"
df = pd.read_csv(csv_file_path)

# Create a table in the database
query = """
    CREATE TABLE IF NOT EXISTS corpus (
        id SERIAL PRIMARY KEY,
        Wikipedia_ID INTEGER,
        Summary TEXT,
        MovieName TEXT,
        Lengte FLOAT,
        Genres TEXT
    );
"""
cursor.execute(query)

for index, row in df.iterrows():
    columns = ', '.join(row.index)
    placeholders = ', '.join(['%s' for _ in range(len(row))])
    insert_query = f"INSERT INTO corpus ({columns}) VALUES ({placeholders});"
    cursor.execute(insert_query, tuple(row))

# Commit the changes and close the cursor and connection
connection.commit()
cursor.close()
connection.close()


Aan de hand van deze code is het gelukt om de data van een csv in een database te krijgen. Nu het in een database staat kan het ingelezen worden met behulp van SQL.

In [5]:
import mysql.connector

host = "sql11.freesqldatabase.com"
database_name = "sql11667769"
database_user = "sql11667769"
database_password = "rMWK4j1uCv"
port_number = "3306"

# Connection to MySQL
connection = mysql.connector.connect(
    host=host,
    database=database_name,
    user=database_user,
    password=database_password,
    port=port_number
)

# Create a cursor
cursor = connection.cursor()

# Fetch all data from the MySQL table into a DataFrame
select_query = "SELECT * FROM corpus;"
cursor.execute(select_query)

# Fetch the column names from the cursor
columns = [column[0] for column in cursor.description]

# Fetch all rows from the cursor
rows = cursor.fetchall()

# Create a DataFrame using the fetched data
df = pd.DataFrame(rows, columns=columns)

# Close the cursor and connection
cursor.close()
connection.close()

# Display the DataFrame
display(df)


,id,wikipediaID,summary,moviename,lengte,genres


## **Bronnen:**

#### Corpus Film Omschrijvingen:

Please cite this paper if you write any papers involving the use of the data above:

    Learning Latent Personas of Film Characters
    David Bamman, Brendan O'Connor, and Noah A. Smith
    ACL 2013, Sofia, Bulgaria, August 2013

